In [1]:
from utils.EWMA import EWMA
from utils.RSI import RSI
from utils.SMA import SMA
from utils.ADX import ADX
from utils.SO import STO
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations
import optuna 

/home/rodo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_5m = pd.read_csv('data/aapl_5m_train.csv')



In [3]:
indicadores = ['EWMA', 'RSI'] #,'SMA', 'ADX', 'STO'

combinaciones_indicadores = []
for i in range(1, len(indicadores) + 1):
    combinaciones_indicadores.extend(combinations(indicadores, i))



In [4]:
combinaciones_indicadores

[('EWMA',), ('RSI',), ('EWMA', 'RSI')]

In [5]:
cash = 1_000_000
active_operations = []
com = 0.00125  # comision en GBM
strategy_value = [1_000_000]

In [6]:
def optimize(trial, strat):
    stop_loss_long = trial.suggest_float('stop_loss_long', 0.01, 0.95)
    take_profit_long = trial.suggest_float('take_profit_long', 0.01, 0.95)
    stop_loss_short = trial.suggest_float('stop_loss_short', 0.01, 0.95)
    take_profit_short = trial.suggest_float('take_profit_short', 0.01, 0.95)
    n_shares = trial.suggest_int('n_shares', 10, 100)
   

    strategy_params = {}

    buy_signals = pd.DataFrame()
    sell_signals = pd.DataFrame()

    if "RSI" in strat:
        strategy_params["RSI"] = {
        "window" : trial.suggest_int('rsi_window', 10, 50),
        "upper_treshold" : trial.suggest_int('rsi_upper_trheshold', 50, 95),
        "lower_treshold" : trial.suggest_int('rsi_lower_trheshold', 5, 49)
        }
        rsi = RSI(df_5m, cash, active_operations, com, strategy_value, n_shares, strategy_params["RSI"]["window"], strategy_params["RSI"]["upper_treshold"], strategy_params["RSI"]["lower_treshold"], stop_loss_long, take_profit_long, stop_loss_short, take_profit_short)
        rsi_buy_signals, rsi_sell_signals, rsi_strategy_value = rsi.run_strategy()
        buy_signals["RSI"] = rsi_buy_signals
        sell_signals["RSI"] = rsi_sell_signals
    if "EWMA" in strat:
        strategy_params["EWMA"] = {
        "long_window" : trial.suggest_int('ewma_long_window', 50, 100),
        "short_window" : trial.suggest_int('ewma_short_window', 5, 49)
        }
        ewma = EWMA(df_5m, cash, active_operations, com, strategy_value, n_shares, strategy_params["EWMA"]["long_window"], strategy_params["EWMA"]["short_window"], stop_loss_long, take_profit_long, stop_loss_short, take_profit_short)
        ewma_buy_signals, ewma_sell_signals, ewma_strategy_value = ewma.run_strategy()
        buy_signals["EWMA"] = ewma_buy_signals
        sell_signals["EWMA"] = ewma_sell_signals

    if "SMA" in strat:
        strategy_params["SMA"] = {
        "long_window" : trial.suggest_int('sma_long_window', 20, 50),
        "short_window" : trial.suggest_int('sma_short_window', 2, 19)
        }
        sma = SMA(df_5m, cash, active_operations, com, strategy_value, n_shares, strategy_params["SMA"]["long_window"], strategy_params["SMA"]["short_window"], stop_loss_long, take_profit_long, stop_loss_short, take_profit_short)
        sma_buy_signals, sma_sell_signals, sma_strategy_value = sma.run_strategy()
        buy_signals["SMA"] = sma_buy_signals
        sell_signals["SMA"] = sma_sell_signals
    
    if "ADX" in strat:
        strategy_params["ADX"] = {
        "window" : trial.suggest_int('adx_window', 10, 50),
        "threshold" : trial.suggest_int('adx_threshold', 15, 30)
        }
        adx = ADX(df_5m, cash, active_operations, com, strategy_value, n_shares, strategy_params["ADX"]["window"], strategy_params["ADX"]["threshold"], stop_loss_long, take_profit_long, stop_loss_short, take_profit_short)
        adx_buy_signals, adx_sell_signals, adx_strategy_value = adx.run_strategy()
        buy_signals["ADX"] = adx_buy_signals
        sell_signals["ADX"] = adx_sell_signals

    if "STO" in strat:
        strategy_params["STO"] = {
        "window" : trial.suggest_int('sto_window', 10, 50),
        "upper_threshold" : trial.suggest_int('sto_upper_threshold', 50, 95),
        "lower_threshold" : trial.suggest_int('sto_lower_threshold', 5, 49)
        }
        sto = STO(df_5m, cash, active_operations, com, strategy_value, n_shares, strategy_params["STO"]["window"], strategy_params["STO"]["upper_threshold"], strategy_params["STO"]["lower_threshold"], stop_loss_long, take_profit_long, stop_loss_short, take_profit_short)
        sto_buy_signals, sto_sell_signals, sto_strategy_value = sto.run_strategy()
        buy_signals["STO"] = sto_buy_signals
        sell_signals["STO"] = sto_sell_signals

    
    # Backtesting
    for i, row in buy_signals.iterrows():
        # If all indicators want to buy...
        if sum(buy_signals.iloc[i,:]) == len(buy_signals.columns):
            buy_signal = True
        else:
            buy_signal = False
        buy_signals.loc[i, "General"] = buy_signal

    for i, row in sell_signals.iterrows():
        # If all indicators want to sell...
        if sum(sell_signals.iloc[i,:]) == len(sell_signals.columns):
            sell_signal = True
        else:
            sell_signal = False
        sell_signals.loc[i, "General"] = sell_signal

    
    return strategy_value[-1]

best_value_overall = float('-inf')
best_params_overall = {}

for strat in combinaciones_indicadores:
    pruner = optuna.pruners.MedianPruner()
    study = optuna.create_study(direction='maximize',sampler=optuna.samplers.TPESampler(), pruner=pruner)
    study.optimize(lambda trial: optimize(trial, strat), n_trials=5, n_jobs=-1)

    # Get the best parameters and their corresponding value
    best_params = study.best_params
    best_value = study.best_value

    if best_value > best_value_overall:
        best_value_overall = best_value
        best_params_overall = best_params

    print("Best parameters for combination:", strat)
    print("Best value:", best_value)
    print("Best parameters:", best_params)
    print()

print("Overall best value:", best_value_overall)
print("Overall best parameters:", best_params_overall)


[I 2024-02-25 14:16:47,070] A new study created in memory with name: no-name-5a832d24-4658-46d8-b3b5-f59e3ba48daf
[I 2024-02-25 14:17:44,675] Trial 0 finished with value: 5993197.182301401 and parameters: {'stop_loss_long': 0.12773071447363823, 'take_profit_long': 0.2855408775584962, 'stop_loss_short': 0.5561746036981263, 'take_profit_short': 0.8133809332434301, 'n_shares': 80, 'ewma_long_window': 78, 'ewma_short_window': 16}. Best is trial 0 with value: 5993197.182301401.
[I 2024-02-25 14:18:20,894] Trial 1 finished with value: 2435654.116177376 and parameters: {'stop_loss_long': 0.5710331361168561, 'take_profit_long': 0.21754186466423941, 'stop_loss_short': 0.05662236231157201, 'take_profit_short': 0.5854770537825043, 'n_shares': 60, 'ewma_long_window': 73, 'ewma_short_window': 11}. Best is trial 0 with value: 5993197.182301401.
[I 2024-02-25 14:19:05,644] Trial 2 finished with value: 3670907.328612167 and parameters: {'stop_loss_long': 0.16080908611681027, 'take_profit_long': 0.4849

Best parameters for combination: ('EWMA',)
Best value: 5993197.182301401
Best parameters: {'stop_loss_long': 0.12773071447363823, 'take_profit_long': 0.2855408775584962, 'stop_loss_short': 0.5561746036981263, 'take_profit_short': 0.8133809332434301, 'n_shares': 80, 'ewma_long_window': 78, 'ewma_short_window': 16}



[I 2024-02-25 14:20:32,104] Trial 0 finished with value: 1950972.509116315 and parameters: {'stop_loss_long': 0.021455869760262987, 'take_profit_long': 0.6693589151214605, 'stop_loss_short': 0.32119130101984067, 'take_profit_short': 0.6438926030872398, 'n_shares': 21, 'rsi_window': 42, 'rsi_upper_trheshold': 64, 'rsi_lower_trheshold': 41}. Best is trial 0 with value: 1950972.509116315.
[I 2024-02-25 14:22:03,330] Trial 1 finished with value: 22828290.915850848 and parameters: {'stop_loss_long': 0.08741674127130425, 'take_profit_long': 0.3814788931044759, 'stop_loss_short': 0.8627481304348449, 'take_profit_short': 0.19459188339195344, 'n_shares': 89, 'rsi_window': 26, 'rsi_upper_trheshold': 61, 'rsi_lower_trheshold': 12}. Best is trial 1 with value: 22828290.915850848.
[I 2024-02-25 14:25:21,510] Trial 2 finished with value: 15260520.380532626 and parameters: {'stop_loss_long': 0.4463890135206167, 'take_profit_long': 0.38939258075361816, 'stop_loss_short': 0.841203586889766, 'take_profi

Best parameters for combination: ('RSI',)
Best value: 22828290.915850848
Best parameters: {'stop_loss_long': 0.08741674127130425, 'take_profit_long': 0.3814788931044759, 'stop_loss_short': 0.8627481304348449, 'take_profit_short': 0.19459188339195344, 'n_shares': 89, 'rsi_window': 26, 'rsi_upper_trheshold': 61, 'rsi_lower_trheshold': 12}



[I 2024-02-25 14:29:40,007] Trial 0 finished with value: 7298504.543449041 and parameters: {'stop_loss_long': 0.46388499276274153, 'take_profit_long': 0.7004517271880379, 'stop_loss_short': 0.5906185927927278, 'take_profit_short': 0.6220683922772062, 'n_shares': 82, 'rsi_window': 19, 'rsi_upper_trheshold': 54, 'rsi_lower_trheshold': 31, 'ewma_long_window': 90, 'ewma_short_window': 16}. Best is trial 0 with value: 7298504.543449041.
[I 2024-02-25 14:30:58,148] Trial 1 finished with value: 8925096.84355918 and parameters: {'stop_loss_long': 0.7852046725153508, 'take_profit_long': 0.32710186374943906, 'stop_loss_short': 0.5202800637673406, 'take_profit_short': 0.6342844835015624, 'n_shares': 94, 'rsi_window': 47, 'rsi_upper_trheshold': 68, 'rsi_lower_trheshold': 46, 'ewma_long_window': 84, 'ewma_short_window': 14}. Best is trial 1 with value: 8925096.84355918.
[I 2024-02-25 14:32:28,434] Trial 2 finished with value: 7309619.466714841 and parameters: {'stop_loss_long': 0.3350046308260049, 

Best parameters for combination: ('EWMA', 'RSI')
Best value: 8925096.84355918
Best parameters: {'stop_loss_long': 0.7852046725153508, 'take_profit_long': 0.32710186374943906, 'stop_loss_short': 0.5202800637673406, 'take_profit_short': 0.6342844835015624, 'n_shares': 94, 'rsi_window': 47, 'rsi_upper_trheshold': 68, 'rsi_lower_trheshold': 46, 'ewma_long_window': 84, 'ewma_short_window': 14}

Overall best value: 22828290.915850848
Overall best parameters: {'stop_loss_long': 0.08741674127130425, 'take_profit_long': 0.3814788931044759, 'stop_loss_short': 0.8627481304348449, 'take_profit_short': 0.19459188339195344, 'n_shares': 89, 'rsi_window': 26, 'rsi_upper_trheshold': 61, 'rsi_lower_trheshold': 12}


ValueError: No trials are completed yet.